# Calculate the CNR rations for effect sizes

Take the outputs of 2-SPM-GLM and make images that takes beta weights and divides by the residual mean squares. I will then take the RMSE error of these images to convert all values to positive.

To check: sub 3 doesn't have a GLM for TR1920

In [2]:
from bids.layout import BIDSLayout
from nipype.interfaces import afni 
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
from nipype.interfaces.utility import Function
import nipype.interfaces.fsl.maths as fsl
from nipype.interfaces import spm as spm
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html

matlab_cmd = '/opt/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

In [3]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
CNRs = pe.Workflow(name="CNRs") # workflow to run the analysis

Plan is to use FLS maths to perform the divisions per GLM, and the RMSE, and save the output in the same folder

# Data grabbers and savers

In [4]:
# THIS CODE DEFINITELY WORKS
dg = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                        outfields=['con','resms']), name='grabber')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'con': [['sub', 'TR']],
                              'resms':[['sub', 'TR']]}
dg.inputs.field_template = {'con': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/TR%s/FLGLM/con_*.nii',
                            'resms': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-%s/TR%s/FLGLM/ResMS.nii'}

In [5]:
# dg.inputs.sub = '03'
# dg.inputs.TR = '1920'
# res = dg.run()
# res.outputs

In [6]:
# THIS WORKS
datinf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                 name='dat-info')
datinf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
#datinf.iterables = [('sub', ['03']), ('TR', ['1920'])]
datinf.iterables 

[('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]

In [7]:
def printSubPath(fullFilePath):
    # given the resms output of the datagrabber, recreate the required filepath for saving the data
    import os
    import re
    fname = os.path.normpath(fullFilePath)
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0], [t for t in l if re.search('TR([0-9])', t)][0]
    name = '/'.join(name)
    return name

# Data sink

In [8]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_ratio([0-9]*)', ''),
                 ('_sqrtsqrs([0-9]*)', '')]
ds.inputs.regexp_substitutions = substitutions

# Zip files 
go from .nii to .nii.gz

In [9]:
from nipype.interfaces.base import (
    TraitedSpec,
    CommandLineInputSpec,
    CommandLine,
    File
)
import os

class GZipInputSpec(CommandLineInputSpec):
    in_file = File(desc="File", exists=True, mandatory=True, argstr="%s")

class GZipOutputSpec(TraitedSpec):
    out_file = File(desc = "Zip file", exists = True)

class GZipTask(CommandLine):
    input_spec = GZipInputSpec
    output_spec = GZipOutputSpec
    _cmd = 'gzip'

    def _list_outputs(self):
            outputs = self.output_spec().get()
            outputs['out_file'] = os.path.abspath(self.inputs.in_file + ".gz")
            return outputs
        


In [10]:
gzipcon = pe.MapNode(GZipTask(), iterfield='in_file', name='gzip')
gzipresms = pe.Node(GZipTask(), name='gzipresms')

# compute square root on single image

In [11]:
sqrtres = pe.Node(fsl.UnaryMaths(), name='sqrt-res')
sqrtres.inputs.operation = 'sqrt'

# Compute the ratio of con to resms
Remember to take the square root of resms!

In [12]:
ratio = pe.MapNode(fsl.BinaryMaths(), name='ratio', iterfield=['in_file', 'out_file'])
ratio.inputs.operation = 'div'

In [13]:
def printCNRName(betaName):
    # this function strips the beta and filepath off the file names from the beta outputs of datagrabber, and renames as ratio
    import os
    fnames = [f.replace('con', 'CNR') for f in betaName ] 
    fnames = [os.path.basename(f) for f in fnames]   
    return fnames

In [14]:
#printCNRName(res.outputs.con)

# iterate and square

In [15]:
sqrratios = pe.MapNode(fsl.UnaryMaths(), name='sqratios', iterfield='in_file')
sqrratios.inputs.operation = 'sqr'

In [16]:
# debug = pe.Workflow(name='debug')
# debug.connect([(datinf, dg, [('sub', 'sub')]),
#                (datinf, dg, [('TR', 'TR')]),
#                (dg, ds, [(('resms', printSubPath), 'container')]),
#                (dg, gzipresms, [('resms', 'in_file')]),
#                (dg, gzipcon, [('con', 'in_file')]),
#                (gzipresms, sqrtres, [('out_file', 'in_file')]),
#                (gzipcon, ratio, [('out_file', 'in_file')]),
#                (sqrtres, ratio, [('out_file', 'operand_file')]),
#                (gzipcon, ratio, [(('out_file', printCNRName), 'out_file')]),
#                (ratio, ds, [('out_file', 'FLGLM.@CNR')])
#              ])
# debug.run()

# iterate and sqrt

In [17]:
sqrtsqrs = pe.MapNode(fsl.UnaryMaths(), name='sqrtsqrs', iterfield=['in_file', 'out_file'])
sqrtsqrs.inputs.operation = 'sqrt'

In [18]:
def printSrSqrtName(betaName):
    # this function strips the beta and filepath off the file names from the beta outputs of datagrabber, and renames as ratio
    import os
    fnames = [f.replace('con', 'RMS_CNR') for f in betaName ] 
    fnames = [os.path.basename(f) for f in fnames]   
    return fnames

In [19]:
#printSrSqrtName(res.outputs.con)

In [20]:
# test.connect([(datinf, dg, [('sub', 'sub')]),
#               (datinf, dg, [('TR', 'TR')]),
#               (dg, ds, [(('resms', printSubPath), 'container')]),
#               (dg, sqrtres, [('resms', 'in_file')]),
#               (dg, ratio, [('beta', 'in_file')]),
#               (sqrtres, ratio, [('out_file', 'operand_file')]),
#               (dg, ratio, [(('beta', printCNRName), 'out_file')]),
#               (ratio, ds, [('out_file', 'FLGLM.@CNR')]),
#               (ratio, sqrratios, [('out_file', 'in_file')]),
#               (sqrratios, sqrtsqrs, [('out_file', 'in_file')]),
#               (dg, sqrtsqrs, [(('beta', printSrSqrtName), 'out_file')]),
#               (sqrtsqrs, ds, [('out_file', 'FLGLM.@RMSE')])
#              ])           
# test.run()

# Connect workflow

In [21]:
CNRs.connect([(datinf, dg, [('sub', 'sub')]),
              (datinf, dg, [('TR', 'TR')]),
              (dg, ds, [(('resms', printSubPath), 'container')]),
              (dg, gzipresms, [('resms', 'in_file')]),
              (dg, gzipcon, [('con', 'in_file')]),
              (gzipresms, sqrtres, [('out_file', 'in_file')]),
              (gzipcon, ratio, [('out_file', 'in_file')]),
              (sqrtres, ratio, [('out_file', 'operand_file')]),
              (gzipcon, ratio, [(('out_file', printCNRName), 'out_file')]),
              (ratio, ds, [('out_file', 'FLGLM.@CNR')]),
              (ratio, sqrratios, [('out_file', 'in_file')]),
              (sqrratios, sqrtsqrs, [('out_file', 'in_file')]),
              (gzipcon, sqrtsqrs, [(('out_file', printSrSqrtName), 'out_file')]),
              (sqrtsqrs, ds, [('out_file', 'FLGLM.@RMSE')])
             ])

In [23]:
CNRs.run()

200914-19:23:31,119 nipype.workflow INFO:
	 Workflow CNRs settings: ['check', 'execution', 'logging', 'monitoring']
200914-19:23:31,233 nipype.workflow INFO:
	 Running serially.
200914-19:23:31,234 nipype.workflow INFO:
	 [Node] Setting-up "CNRs.grabber" in "/tmp/tmpc21jn6it/CNRs/_TR_1920_sub_05/grabber".
200914-19:23:31,240 nipype.workflow INFO:
	 [Node] Running "grabber" ("nipype.interfaces.io.DataGrabber")
200914-19:23:31,245 nipype.workflow INFO:
	 [Node] Finished "CNRs.grabber".
200914-19:23:31,246 nipype.workflow INFO:
	 [Node] Setting-up "CNRs.gzip" in "/tmp/tmp59pqlohc/CNRs/_TR_1920_sub_05/gzip".
200914-19:23:31,252 nipype.workflow INFO:
	 [Node] Setting-up "_gzip0" in "/tmp/tmp59pqlohc/CNRs/_TR_1920_sub_05/gzip/mapflow/_gzip0".
200914-19:23:31,257 nipype.workflow INFO:
	 [Node] Running "_gzip0" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/con_0001.nii
200914-19:23:31,465 nipype.workflow INFO

200914-19:23:35,222 nipype.workflow INFO:
	 [Node] Finished "_ratio6".
200914-19:23:35,228 nipype.workflow INFO:
	 [Node] Finished "CNRs.ratio".
200914-19:23:35,228 nipype.workflow INFO:
	 [Node] Setting-up "CNRs.sqratios" in "/tmp/tmpngym1ko1/CNRs/_TR_1920_sub_05/sqratios".
200914-19:23:35,236 nipype.workflow INFO:
	 [Node] Setting-up "_sqratios0" in "/tmp/tmpngym1ko1/CNRs/_TR_1920_sub_05/sqratios/mapflow/_sqratios0".
200914-19:23:35,241 nipype.workflow INFO:
	 [Node] Running "_sqratios0" ("nipype.interfaces.fsl.maths.UnaryMaths"), a CommandLine Interface with command:
fslmaths /tmp/tmp6e9jvj6g/CNRs/_TR_1920_sub_05/ratio/mapflow/_ratio0/CNR_0001.nii.gz -sqr /tmp/tmpngym1ko1/CNRs/_TR_1920_sub_05/sqratios/mapflow/_sqratios0/CNR_0001_sqr.nii.gz
200914-19:23:35,445 nipype.workflow INFO:
	 [Node] Finished "_sqratios0".
200914-19:23:35,446 nipype.workflow INFO:
	 [Node] Setting-up "_sqratios1" in "/tmp/tmpngym1ko1/CNRs/_TR_1920_sub_05/sqratios/mapflow/_sqratios1".
200914-19:23:35,451 nipype

200914-19:23:38,252 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/_ratio1/CNR_0002.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM///CNR_0002.nii.gz
200914-19:23:38,262 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/_ratio2/CNR_0003.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM///CNR_0003.nii.gz
200914-19:23:38,271 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/_ratio3/CNR_0005.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM///CNR_0005.nii.gz
200914-19:23:38,280 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM/_TR_1920_sub_05/_ratio4/CNR_0006.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1920/FLGLM///CNR_0006.nii.gz
200914-19:23:38,289 nipype.i

200914-19:23:40,729 nipype.workflow INFO:
	 [Node] Finished "_ratio0".
200914-19:23:40,732 nipype.workflow INFO:
	 [Node] Setting-up "_ratio1" in "/tmp/tmppnlyavmb/CNRs/_TR_1510_sub_05/ratio/mapflow/_ratio1".
200914-19:23:40,737 nipype.workflow INFO:
	 [Node] Running "_ratio1" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/FLGLM/con_0002.nii.gz -div /tmp/tmp9540buow/CNRs/_TR_1510_sub_05/sqrt-res/ResMS_sqrt.nii.gz CNR_0002.nii.gz
200914-19:23:40,980 nipype.workflow INFO:
	 [Node] Finished "_ratio1".
200914-19:23:40,982 nipype.workflow INFO:
	 [Node] Setting-up "_ratio2" in "/tmp/tmppnlyavmb/CNRs/_TR_1510_sub_05/ratio/mapflow/_ratio2".
200914-19:23:40,987 nipype.workflow INFO:
	 [Node] Running "_ratio2" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-05/TR1510/FLGLM/con_0003.nii.gz -div /tmp/

200914-19:23:44,45 nipype.workflow INFO:
	 [Node] Running "_sqrtsqrs1" ("nipype.interfaces.fsl.maths.UnaryMaths"), a CommandLine Interface with command:
fslmaths /tmp/tmpjzatv_vv/CNRs/_TR_1510_sub_05/sqratios/mapflow/_sqratios1/CNR_0002_sqr.nii.gz -sqrt RMS_CNR_0002.nii.gz
200914-19:23:44,242 nipype.workflow INFO:
	 [Node] Finished "_sqrtsqrs1".
200914-19:23:44,244 nipype.workflow INFO:
	 [Node] Setting-up "_sqrtsqrs2" in "/tmp/tmpaz4hd_0g/CNRs/_TR_1510_sub_05/sqrtsqrs/mapflow/_sqrtsqrs2".
200914-19:23:44,249 nipype.workflow INFO:
	 [Node] Running "_sqrtsqrs2" ("nipype.interfaces.fsl.maths.UnaryMaths"), a CommandLine Interface with command:
fslmaths /tmp/tmpjzatv_vv/CNRs/_TR_1510_sub_05/sqratios/mapflow/_sqratios2/CNR_0003_sqr.nii.gz -sqrt RMS_CNR_0003.nii.gz
200914-19:23:44,468 nipype.workflow INFO:
	 [Node] Finished "_sqrtsqrs2".
200914-19:23:44,471 nipype.workflow INFO:
	 [Node] Setting-up "_sqrtsqrs3" in "/tmp/tmpaz4hd_0g/CNRs/_TR_1510_sub_05/sqrtsqrs/mapflow/_sqrtsqrs3".
200914-19

...

200914-19:24:55,718 nipype.workflow INFO:
	 [Node] Running "grabber" ("nipype.interfaces.io.DataGrabber")
200914-19:24:55,722 nipype.workflow INFO:
	 [Node] Finished "CNRs.grabber".
200914-19:24:55,723 nipype.workflow INFO:
	 [Node] Setting-up "CNRs.gzip" in "/tmp/tmpg0tcj7j3/CNRs/_TR_1510_sub_01/gzip".
200914-19:24:55,729 nipype.workflow INFO:
	 [Node] Setting-up "_gzip0" in "/tmp/tmpg0tcj7j3/CNRs/_TR_1510_sub_01/gzip/mapflow/_gzip0".
200914-19:24:55,733 nipype.workflow INFO:
	 [Node] Running "_gzip0" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM/con_0001.nii
200914-19:24:55,913 nipype.workflow INFO:
	 [Node] Finished "_gzip0".
200914-19:24:55,914 nipype.workflow INFO:
	 [Node] Setting-up "_gzip1" in "/tmp/tmpg0tcj7j3/CNRs/_TR_1510_sub_01/gzip/mapflow/_gzip1".
200914-19:24:55,918 nipype.workflow INFO:
	 [Node] Running "_gzip1" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch

200914-19:24:59,612 nipype.workflow INFO:
	 [Node] Setting-up "CNRs.sqratios" in "/tmp/tmp3_d15f6h/CNRs/_TR_1510_sub_01/sqratios".
200914-19:24:59,619 nipype.workflow INFO:
	 [Node] Setting-up "_sqratios0" in "/tmp/tmp3_d15f6h/CNRs/_TR_1510_sub_01/sqratios/mapflow/_sqratios0".
200914-19:24:59,623 nipype.workflow INFO:
	 [Node] Running "_sqratios0" ("nipype.interfaces.fsl.maths.UnaryMaths"), a CommandLine Interface with command:
fslmaths /tmp/tmpz8wgs715/CNRs/_TR_1510_sub_01/ratio/mapflow/_ratio0/CNR_0001.nii.gz -sqr /tmp/tmp3_d15f6h/CNRs/_TR_1510_sub_01/sqratios/mapflow/_sqratios0/CNR_0001_sqr.nii.gz
200914-19:24:59,877 nipype.workflow INFO:
	 [Node] Finished "_sqratios0".
200914-19:24:59,878 nipype.workflow INFO:
	 [Node] Setting-up "_sqratios1" in "/tmp/tmp3_d15f6h/CNRs/_TR_1510_sub_01/sqratios/mapflow/_sqratios1".
200914-19:24:59,883 nipype.workflow INFO:
	 [Node] Running "_sqratios1" ("nipype.interfaces.fsl.maths.UnaryMaths"), a CommandLine Interface with command:
fslmaths /tmp/tmp

200914-19:25:02,926 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM/_TR_1510_sub_01/_ratio1/CNR_0002.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM///CNR_0002.nii.gz
200914-19:25:02,937 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM/_TR_1510_sub_01/_ratio2/CNR_0003.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM///CNR_0003.nii.gz
200914-19:25:02,947 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM/_TR_1510_sub_01/_ratio3/CNR_0005.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM///CNR_0005.nii.gz
200914-19:25:02,957 nipype.interface INFO:
	 sub: /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM/_TR_1510_sub_01/_ratio4/CNR_0006.nii.gz -> /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR1510/FLGLM///CNR_0006.nii.gz
200914-19:25:02,967 nipype.i

200914-19:25:04,631 nipype.workflow INFO:
	 [Node] Finished "_ratio0".
200914-19:25:04,633 nipype.workflow INFO:
	 [Node] Setting-up "_ratio1" in "/tmp/tmpgqvo270n/CNRs/_TR_700_sub_01/ratio/mapflow/_ratio1".
200914-19:25:04,638 nipype.workflow INFO:
	 [Node] Running "_ratio1" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM/con_0002.nii.gz -div /tmp/tmpe2fbus4d/CNRs/_TR_700_sub_01/sqrt-res/ResMS_sqrt.nii.gz CNR_0002.nii.gz
200914-19:25:04,814 nipype.workflow INFO:
	 [Node] Finished "_ratio1".
200914-19:25:04,817 nipype.workflow INFO:
	 [Node] Setting-up "_ratio2" in "/tmp/tmpgqvo270n/CNRs/_TR_700_sub_01/ratio/mapflow/_ratio2".
200914-19:25:04,822 nipype.workflow INFO:
	 [Node] Running "_ratio2" ("nipype.interfaces.fsl.maths.BinaryMaths"), a CommandLine Interface with command:
fslmaths /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glm/sub-01/TR700/FLGLM/con_0003.nii.gz -div /tmp/tmpe2

200914-19:25:07,236 nipype.workflow INFO:
	 [Node] Finished "_sqrtsqrs1".
200914-19:25:07,238 nipype.workflow INFO:
	 [Node] Setting-up "_sqrtsqrs2" in "/tmp/tmpbzdbg0ii/CNRs/_TR_700_sub_01/sqrtsqrs/mapflow/_sqrtsqrs2".
200914-19:25:07,243 nipype.workflow INFO:
	 [Node] Running "_sqrtsqrs2" ("nipype.interfaces.fsl.maths.UnaryMaths"), a CommandLine Interface with command:
fslmaths /tmp/tmpos9rxigh/CNRs/_TR_700_sub_01/sqratios/mapflow/_sqratios2/CNR_0003_sqr.nii.gz -sqrt RMS_CNR_0003.nii.gz
200914-19:25:07,429 nipype.workflow INFO:
	 [Node] Finished "_sqrtsqrs2".
200914-19:25:07,431 nipype.workflow INFO:
	 [Node] Setting-up "_sqrtsqrs3" in "/tmp/tmpbzdbg0ii/CNRs/_TR_700_sub_01/sqrtsqrs/mapflow/_sqrtsqrs3".
200914-19:25:07,435 nipype.workflow INFO:
	 [Node] Running "_sqrtsqrs3" ("nipype.interfaces.fsl.maths.UnaryMaths"), a CommandLine Interface with command:
fslmaths /tmp/tmpos9rxigh/CNRs/_TR_700_sub_01/sqratios/mapflow/_sqratios3/CNR_0005_sqr.nii.gz -sqrt RMS_CNR_0005.nii.gz
200914-19:25